<a href="https://colab.research.google.com/github/JasonCFowler/DigitalRepos/blob/master/MARC_CSV_Merge_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
# Section 1: Install Pymarc
# -------------------------
!pip install pymarc
# Section 2: Import Required Libraries
# ------------------------------------
import csv
from pymarc import MARCReader, Field, Subfield
import pymarc

In [37]:
# Section 3: Define Function to Add 949 Field
# -------------------------------------------
def add_949_to_marc(marc_record, data_from_csv):
    # Subfield h is the OCLC branch, subfield l is the shelving location,
    # and subfield p is the fund code
    field = pymarc.Field(
        tag='949',
        indicators=[' ', ' '],
        subfields=[
            pymarc.Subfield(code='h', value='5053'),
            pymarc.Subfield(code='l', value='Unspecified'),
            pymarc.Subfield(code='p', value=data_from_csv.get('FundCode', ' '))
        ]
    )
    marc_record.add_field(field)

In [38]:
# Section 4: Define Function to Add 980 Field
# -------------------------------------------
def add_980_to_marc(marc_record, data_from_csv):
    # Subfield b is the unit price, subfield c is the service charge,
    # subfield h is the tax percentage, subfield i is the discount percentage,
    # subfield k is the isbn, and subfield z is the currency of the purchase order
    field = pymarc.Field(
        tag='980',
        indicators=[' ', ' '],
        subfields=[
            pymarc.Subfield(code='b', value=data_from_csv.get('UnitPrice', ' ')),
            pymarc.Subfield(code='i', value=data_from_csv.get('DiscountPercentage', ' ')),
            pymarc.Subfield(code='k', value=data_from_csv.get('ISBN', ' ')),
            pymarc.Subfield(code='z', value='USD')
        ]
    )
    marc_record.add_field(field)

In [39]:
# Section 5: Define Function to Add 981 Field
# -------------------------------------------
def add_981_to_marc(marc_record, data_from_csv):
    # Subfield a is the KB Collection Identifier,
    # subfield b is the KB Title Identifier, and
    # subfield n is the Vendor Order Number
    field = pymarc.Field(
        tag='981',
        indicators=[' ', ' '],
        subfields=[
            pymarc.Subfield(code='a', value=data_from_csv.get('KB_Collection_ID', ' ')),
            pymarc.Subfield(code='b', value=data_from_csv.get('KB_Title_ID', ' ')),
            pymarc.Subfield(code='n', value=data_from_csv.get('VendorOrderNumber', ' '))
        ]
    )
    marc_record.add_field(field)


In [40]:
# Section 6: Function to match and update MARC records based on ISBN
def update_marc_records(marc_file_path, csv_file_path):
    # Read MARC records
    with open(marc_file_path, 'rb') as marc_file:
        reader = MARCReader(marc_file)
        records = list(reader)

    # Read CSV data
    with open(csv_file_path, 'r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            isbn = row.get('ISBN', row.get('\ufeffISBN'))  # Modified this line

            # Match MARC record using ISBN
            matched_record = None
            for record in records:
                # Add your matching criteria here
                isbns = [field['a'] for field in record.get_fields('020') if 'a' in field]
                print(f"MARC Record ISBNs: {isbns}")
                print(f"ISBN to Match: {isbn}")
                if any(isbn == record_isbn for record_isbn in isbns):
                    matched_record = record
                    break

            # Update MARC record with 949, 980, and 981 fields
            if matched_record:
                print(f"Processing ISBN: {isbn}")
                try:
                    add_949_to_marc(matched_record, row)
                    add_980_to_marc(matched_record, row)
                    add_981_to_marc(matched_record, row)
                except KeyError as e:
                    print(f"KeyError: {e}")

    # Write updated MARC records to a new file
    with open('updated_marc_records.mrc', 'wb') as updated_marc_file:
        for record in records:
            updated_marc_file.write(record.as_marc())


In [ ]:
# Section 7: Execute
if __name__ == "__main__":
    # Specify file paths
    marc_file_path = '/content/CombinedOrders.mrc'
    csv_file_path = '/content/your_csv_file-test-processed202311230752.csv'

    # Call the function to update MARC records
    update_marc_records(marc_file_path, csv_file_path)


In [42]:
# Print the first few rows of the CSV file
# This is used for troubleshooting the CSV file
# It's not part of the actual code

with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    for i, row in enumerate(csv_reader):
        print(row)
        if i == 5:  # Print only the first 5 rows for inspection
            break

['ISBN', 'KB_Collection_ID', 'ShelvingLocation', 'FundCode', 'UnitPrice', 'ServiceCharge', 'TaxPercentage', 'DiscountPercentage', 'KB_Title_ID', 'Vendor_Order_Number', 'Title', 'Author', 'Date', 'Publisher']
['9781467463102', 'netlibrary.ebooks', 'Unspecified', 'EBOOKS OTHER', '$59.99', '', '', '', '2274fd7b-8049-4b30-922e-0a8e95046b01-emi', '', 'The Incomparable God', '', '', '']
['9780830855162', 'netlibrary.ebooks', 'Unspecified', 'EBOOKS OTHER', '$32.49', '', '', '', '600dff17-b2f1-4f69-bd00-dd5408b350ba-emi', '', 'How Do We Reason?', '', '', '']
['9781433559945', 'netlibrary.ebooks', 'Unspecified', 'EBOOKS OTHER', '$49.99', '', '', '', 'e729a5a9-88c6-48fa-948b-7e894e3b16e0-emi', '', 'Reformed Systematic Theology, Volume 3', '', '', '']
['9781467462297', 'netlibrary.ebooks', 'Unspecified', 'EBOOKS OTHER', '$50.00', '', '', '', '8055af33-0a9a-44bc-b2e5-6cae242ed12b-emi', '', 'A Companion to the Theology of John Webster', '', '', '']
['9780310124313', 'netlibrary.ebooks', 'Unspecifie